In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import gzip
import pickle
import torch

# Imágenes astronómicas

Las imágenes astrónomicas usualmente se guardan en formato FITS

Podemos usar la librería [astropy](https://docs.astropy.org/en/stable/io/fits/) para abrir y manipular estos archivos

El repositorio tiene una imagen de ejemplo en la carpeta `../data`

Algunos atributos importantes son
- `info()` para explorar el contenido del archivo
- `header` para recuperar los parámetros que se usaron para producir la imagen y otra metadata
- `data` para recuperar la imagen en si

Como la imagen tiene "valores extremos" usamos visualizamos su logaritmo

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
with gzip.open('../data/example.fits.gz') as f:
    with fits.open(f) as hdul:
        display(hdul.info())
        #print(repr(hdul[0].header))
        imap = ax.matshow(np.log(1+hdul[0].data).T, cmap=plt.cm.Greys_r)
        fig.colorbar(imap, ax=ax)

# Detección de transientes y variables con resta de imágenes

<img src="../img/image-subtraction.png" width="800">

<img src="../img/image-candidates.png" width="800">

<img src="../img/image-pipeline.png" width="800">

Usaremos un conjunto de candidatos a transiente obtenidos luego del paso de sustración de imágenes durante el proyecto High Cadence Transient Survey (HiTS) el año 2013

Los candidatos están etiquetados como

- 0 : Candidato real (dominado por artefactos)
- 1 : Candidato sintético insertado en la pipeline

Se [insertaron candidatos sintéticos](https://iopscience.iop.org/article/10.1086/519832/meta) para balancear el dataset y entrenar un clasificador, originalmente un [random forest](https://iopscience.iop.org/article/10.3847/0004-637X/832/2/155/meta) y luego una [red neuronal convolucional](https://iopscience.iop.org/article/10.3847/1538-4357/836/1/97/pdf)

Cada candidato está representado por tres imágenes de 21x21 pixeles

- 0: Imagen de diferencia
- 1: Imagen de ciencia
- 2: Imagen de referencia

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, Subset 

with gzip.open("../data/images2.pgz", mode="r") as f:
    astro_image, astro_label = pickle.load(f)

# Reescalamiento a [0, 1]
astro_image_tensor = torch.from_numpy(astro_image.astype('float32')).reshape(-1, 3, 21, 21)
im_min = astro_image_tensor.min(dim=-1).values.min(dim=-1).values.reshape(-1, 3, 1, 1)
im_max = astro_image_tensor.max(dim=-1).values.max(dim=-1).values.reshape(-1, 3, 1, 1)
astro_image_tensor = (astro_image_tensor - im_min)/(im_max-im_min)

# Creación de DataSet y DataLoader
astro_dataset = TensorDataset(astro_image_tensor, torch.from_numpy(astro_label))

astro_loader = DataLoader(astro_dataset, 
                          batch_size=128, 
                          shuffle=True)

In [ ]:
for x, y in astro_loader:
    break
    
fig, ax = plt.subplots(1, 10, figsize=(9, 1.5), tight_layout=True)
for axi, xi, yi in zip(ax, x, y):
    axi.imshow(xi[0], cmap=plt.cm.Greys_r)
    axi.axis('off')
    axi.set_title(yi.item())


El conjunto completo está en `/home/shared/astro/HiTS/HiTS_500k/images_train.csv` como archivo csv

# Tarea final Unidad 5

- Entrenar un GAN con **las imágenes de diferencia** de HiTS
- Reescale las imágenes al rango $[0, 1]$
- Utilice como base la arquitectura [DCGAN](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)
- Son libres de experimentar, se premiará la creatividad
- Se trabajará en grupos de a dos
- Recomiendo usar guanaco para hacer los entrenamientos en GPU. Puede usar el ambiente de conda `astro`